In [ ]:
from src.hpe.mp.performance import EstimationCollector

collector = EstimationCollector()
df = collector.collect(name="estimations")

In [ ]:
from src.hpe.yolo.performance import EstimationCollector as YoloEstimationCollector

collector = YoloEstimationCollector()
df_yolo = collector.collect(name="estimations")

In [1]:
from src.hpe.mp.performance import read_estimations as read_mp_estimations
from src.hpe.yolo.performance import read_estimations as read_yolo_estimations

df_mp = read_mp_estimations()
df_yolo = read_yolo_estimations()

2025-10-07 15:30:14.863562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759843814.881439  763114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759843814.886668  763114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-07 15:30:14.904927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


loaded labels


In [2]:
from src.hpe.common.performance import HpeEstimation

df_mp_est = df_mp.map(HpeEstimation.prediction_result)

In [6]:
from pandas import Series

df_counts = df_mp_est.apply(Series.value_counts).fillna(0)
df_totals = df_counts.apply(sum, axis=1)

In [7]:
df_totals["FN"]

636.0

In [8]:
from pandas import DataFrame

def precision(totals: DataFrame) -> float:
    return totals["TP"]  / (totals["TP"] + totals["FP"])

def recall(totals: DataFrame) -> float:
    return totals["TP"]  / (totals["TP"] + totals["FN"])

In [9]:
print(f"Precision: {precision(df_totals)}")
print(f"Recall: {recall(df_totals)}")

Precision: 0.6755952380952381
Recall: 0.2630359212050985


In [ ]:
from src.hpe.yolo.performance import read_distances as read_yolo_distances
from src.hpe.mp.performance import read_distances as read_mp_distances
from src.hpe.common.plot import plot_distances_boxplot

mp_dist = read_mp_distances(dataset_name="distances")
mp_bgr_dist = read_mp_distances(dataset_name="distances_bgr")

yolo_dist = read_yolo_distances(dataset_name="distances")
yolo_bgr_dist = read_yolo_distances(dataset_name="distances_bgr")

In [ ]:
plot_distances_boxplot(
    None,
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
plot_distances_boxplot(
    (0, 10),
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation
from src.sota.kfold import SOTAFoldCrossValidation
from src.common.plot import box_plot_accuracies

box_plot_accuracies(
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-unbalanced-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-dr0.3-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-yolo-kf"),
    # HpeDnnFoldCrossValidation.evaluation_instance("arch2-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch3-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch4-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch5-kf"),
    SOTAFoldCrossValidation.evaluation_instance("yolo11n-kf", "yolov11n-cls"))